# Data Preprocessing

## 1. Introduction

In this notebook, we will preprocess the data to make it ready for the model. We will focus on data cleaning, integration, and aggregation. We will discuss methodologies investigated and chosen for handling of missing values, feature engineering, and feature selection.

## 2. Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns

# Load the environment variables
load_dotenv("../config/.env")

scripts_path = os.getenv("SCRIPTS_PATH")

# Add the path to the scripts folder and import the functions
if scripts_path is not None:
    if scripts_path not in sys.path:
        sys.path.append(scripts_path)

# Import the functions
plt.style.use('ggplot')
pd.set_option('display.max_columns', None)

In [2]:
# from raw_data import get_raw_dataframes function
from raw_data import get_raw_dataframes

## 3. Load Data

In [3]:
# Load data
city_data, country_data, weather_data, migraine_data = get_raw_dataframes()

# Check the shape of the dataframes
city_data.shape, country_data.shape, weather_data.shape, migraine_data.shape

((1245, 8), (214, 11), (27635763, 14), (1377000, 10))

## 4. Data Cleaning

### 4.1 Drop Unnecessary Columns/Rows

#### 4.1.1 DataFrame: `city_data`

In [4]:
# View the `city_data` dataframe
city_data

,station_id,city_name,country,state,iso2,iso3,latitude,longitude
0,41515,Asadabad,Afghanistan,Kunar,AF,AFG,34.866000,71.150005
1,38954,Fayzabad,Afghanistan,Badakhshan,AF,AFG,37.129761,70.579247
2,41560,Jalalabad,Afghanistan,Nangarhar,AF,AFG,34.441527,70.436103
3,38947,Kunduz,Afghanistan,Kunduz,AF,AFG,36.727951,68.872530
4,38987,Qala i Naw,Afghanistan,Badghis,AF,AFG,34.983000,63.133300
...,...,...,...,...,...,...,...,...
1240,67475,Kasama,Zambia,Northern,ZM,ZMB,-10.199598,31.179947
1241,68030,Livingstone,Zambia,Southern,ZM,ZMB,-17.860009,25.860013
1242,67633,Mongu,Zambia,Western,ZM,ZMB,-15.279598,23.120025
1243,67775,Harare,Zimbabwe,Harare,ZW,ZWE,-17.817790,31.044709


Keeping all columns for now.

#### 4.1.2 DataFrame: `country_data`

In [5]:
# View the `country_data` dataframe
country_data

,country,native_name,iso2,iso3,population,area,capital,capital_lat,capital_lng,region,continent
0,Afghanistan,افغانستان,AF,AFG,26023100.0,652230.0,Kabul,34.526011,69.177684,Southern and Central Asia,Asia
1,Albania,Shqipëria,AL,ALB,2895947.0,28748.0,Tirana,41.326873,19.818791,Southern Europe,Europe
2,Algeria,الجزائر,DZ,DZA,38700000.0,2381741.0,Algiers,36.775361,3.060188,Northern Africa,Africa
3,American Samoa,American Samoa,AS,ASM,55519.0,199.0,Pago Pago,-14.275479,-170.704830,Polynesia,Oceania
4,Angola,Angola,AO,AGO,24383301.0,1246700.0,Luanda,-8.827270,13.243951,Central Africa,Africa
...,...,...,...,...,...,...,...,...,...,...,...
209,Wallis and Futuna,Wallis et Futuna,WF,WLF,13135.0,142.0,Mata-Utu,-13.282042,-176.174022,Polynesia,Oceania
210,Western Sahara,الصحراء الغربية,EH,ESH,586000.0,266000.0,El Aaiún,27.154512,-13.195392,Northern Africa,Africa
211,Yemen,اليَمَن,YE,YEM,25956000.0,527968.0,Sana'a,15.353857,44.205884,Middle East,Asia
212,Zambia,Zambia,ZM,ZMB,15023315.0,752612.0,Lusaka,-15.416449,28.282154,Eastern Africa,Europe


*Keeping* the following columns:
- 'country'
- 'iso2'
- 'iso3'
- 'population'
- 'area'

*Removing* the following columns:
- 'native_name'
- 'capital'
- 'capital_lat'
- 'capital_lng'
- 'region'
- 'continent'

In [6]:
# Drop columns that are not needed for the analysis
country_data.drop(columns=['native_name', 'capital', 'capital_lat', 'capital_lng', 'region', 'continent'], inplace=True)

# Check the shape of the dataframe
country_data.shape

(214, 5)

#### 4.1.3 DataFrame: `weather_data`

In [7]:
# View the `weather_data` dataframe
weather_data

,station_id,city_name,date,season,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,snow_depth_mm,avg_wind_dir_deg,avg_wind_speed_kmh,peak_wind_gust_kmh,avg_sea_level_pres_hpa,sunshine_total_min
0,41515,Asadabad,1957-07-01,Summer,27.0,21.1,35.6,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,41515,Asadabad,1957-07-02,Summer,22.8,18.9,32.2,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,41515,Asadabad,1957-07-03,Summer,24.3,16.7,35.6,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,41515,Asadabad,1957-07-04,Summer,26.6,16.1,37.8,4.1,NaN,NaN,NaN,NaN,NaN,NaN
4,41515,Asadabad,1957-07-05,Summer,30.8,20.0,41.7,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24216,67975,Masvingo,2023-09-01,Spring,19.5,9.6,28.4,NaN,NaN,180.0,4.6,NaN,NaN,NaN
24217,67975,Masvingo,2023-09-02,Spring,21.3,10.5,31.4,NaN,NaN,146.0,6.3,NaN,NaN,NaN
24218,67975,Masvingo,2023-09-03,Spring,22.1,13.0,31.5,NaN,NaN,147.0,8.2,NaN,NaN,NaN
24219,67975,Masvingo,2023-09-04,Spring,21.5,13.1,29.7,NaN,NaN,155.0,10.2,NaN,NaN,NaN


*Keeping* all columns:
- 'station_id'
- 'city_name'
- 'date'
- 'season'
- '*_temp_c' (avg, min, max)
- 'precipitation_mm'
- 'snow_depth_mm'
- 'avg_wind_dir_deg'
- 'avg_wind_speed_kmh'
- 'peak_wind_gust_kmh'
- 'avg_sea_level_pres_hpa'
- 'sunshine_total_min'

#### 4.1.4 DataFrame: `migraine_data`

In [8]:
# View the migraine data
migraine_data

,measure,location,sex,age,cause,metric,year,val,upper,lower
0,Prevalence,Albania,Male,<5 years,Migraine,Number,1990,0.000000,0.000000,0.000000
1,Prevalence,Albania,Female,<5 years,Migraine,Number,1990,0.000000,0.000000,0.000000
2,Prevalence,Albania,Male,<5 years,Migraine,Percent,1990,0.000000,0.000000,0.000000
3,Prevalence,Albania,Female,<5 years,Migraine,Percent,1990,0.000000,0.000000,0.000000
4,Prevalence,Albania,Male,<5 years,Migraine,Rate,1990,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
376995,Prevalence,Texas,Female,85+ years,Tension-type headache,Number,2019,74932.802890,97941.433283,55430.840013
376996,Prevalence,Texas,Male,85+ years,Tension-type headache,Percent,2019,0.262642,0.345004,0.190791
376997,Prevalence,Texas,Female,85+ years,Tension-type headache,Percent,2019,0.297090,0.388295,0.219764
376998,Prevalence,Texas,Male,85+ years,Tension-type headache,Rate,2019,26262.174931,34497.022166,19078.504371


*Keeping* all columns:
- 'measure'
- 'location'
- 'sex'
- 'age'
- 'cause'
- 'metric'
- 'year'
- 'val'
- 'upper'
- 'lower'

In [9]:
# Check the unique values of the measure_name column
migraine_data['measure'].unique()

array(['Prevalence'], dtype=object)

In [10]:
# Check the unique values of the metric_name column
migraine_data['metric'].unique()

array(['Number', 'Percent', 'Rate'], dtype=object)

For this analysis, we will only be looking at the prevalence (total # of cases in the population) of headache disorders (i.e., migraine, tension-type headache) and confirmed no other measure name columns are present in our dataset. We will remove all percent and rate values, as we are only interested in the total number of cases.

In [11]:
# Filter the data to only include the number of headache and migraine cases

# Identify indices to drop for both 'metric'
metric_indices_to_drop = migraine_data[migraine_data['metric'].isin(['Percent', 'Rate'])].index

# Drop rows
filtered_migraine_data = migraine_data.drop(metric_indices_to_drop)

# Format values in the val, upper, and lower columns to two decimal places
pd.set_option('display.float_format', lambda x:'%.2f' % x)

filtered_migraine_data

,measure,location,sex,age,cause,metric,year,val,upper,lower
3415,Prevalence,Taiwan (Province of China),Male,85+ years,Migraine,Number,2011,5249.38,6903.76,3990.16
3421,Prevalence,Taiwan (Province of China),Male,85+ years,Migraine,Number,2012,5618.06,7341.49,4266.90
3427,Prevalence,Taiwan (Province of China),Male,85+ years,Migraine,Number,2013,5999.05,7848.98,4565.28
3433,Prevalence,Taiwan (Province of China),Male,85+ years,Migraine,Number,2014,6424.09,8403.95,4870.36
3439,Prevalence,Taiwan (Province of China),Male,85+ years,Migraine,Number,2015,6721.02,8764.60,5091.28
...,...,...,...,...,...,...,...,...,...,...
376485,Prevalence,Texas,Female,60-64 years,Tension-type headache,Number,1994,136301.30,177640.61,99142.23
376490,Prevalence,Texas,Male,60-64 years,Tension-type headache,Number,1995,111548.80,148253.99,79929.00
376491,Prevalence,Texas,Female,60-64 years,Tension-type headache,Number,1995,136406.47,177886.24,99248.86
376496,Prevalence,Texas,Male,60-64 years,Tension-type headache,Number,1996,112390.22,149308.16,80564.24


### 4.2 Review and Plan for Missing/Zero Values

#### 4.2.1 DataFrame: `city_data`

In [12]:
# Check for missing values
print("\nCity Missing Values:\n")
print(city_data.isnull().sum())

# Calculate zero counts for each column
print("\nCity Zero Counts:\n")
zero_counts = (city_data == 0).sum()
print(zero_counts)


City Missing Values:

station_id     0
city_name      1
country        0
state         28
iso2           6
iso3           0
latitude       0
longitude      0
dtype: int64

City Zero Counts:

station_id    0
city_name     0
country       0
state         0
iso2          0
iso3          0
latitude      0
longitude     0
dtype: int64


Plan: Merge with `country_data` then merge with `weather_data` to fill in missing values.

#### 4.2.2 DataFrame: `country_data`

In [13]:
# Check for missing values
print("\nCountry Missing Values:\n")
print(country_data.isnull().sum())

# Calculate zero counts for each column
print("\nCountry Zero Counts:\n")
zero_counts = (country_data == 0).sum()
print(zero_counts)


Country Missing Values:

country       0
iso2          1
iso3          0
population    4
area          7
dtype: int64

Country Zero Counts:

country       0
iso2          0
iso3          0
population    0
area          0
dtype: int64


Plan: Merge with `city_data` then merge with `weather_data` to fill in missing values.

#### 4.2.3 DataFrame: `weather_data`

In [14]:
# Check for missing values
print("\nWeather Missing Values:\n")
print(weather_data.isnull().sum())

# Calculate zero counts for each column
print("\nWeather Zero Counts:\n")
zero_counts = (weather_data == 0).sum()
print(zero_counts)


Weather Missing Values:

station_id                       0
city_name                    13993
date                             0
season                           0
avg_temp_c                 6230907
min_temp_c                 5718229
max_temp_c                 5539346
precipitation_mm           6642500
snow_depth_mm             24208615
avg_wind_dir_deg          24183195
avg_wind_speed_kmh        22350295
peak_wind_gust_kmh        26514277
avg_sea_level_pres_hpa    23618606
sunshine_total_min        26614302
dtype: int64

Weather Zero Counts:

station_id                       0
city_name                        0
date                             0
season                           0
avg_temp_c                   26378
min_temp_c                  156684
max_temp_c                   57284
precipitation_mm          13381259
snow_depth_mm              2435637
avg_wind_dir_deg             11758
avg_wind_speed_kmh            5137
peak_wind_gust_kmh            3476
avg_sea_level_pres_hpa      

Plan: Merge with `city_data` and `country_data` to fill in missing values. Then recheck missing and zero counts after merging.

#### 4.2.4 DataFrame: `filtered_migraine_data`

In [15]:
# Check for missing values
print("\nFiltered Migraine Missing Values:\n")
print(filtered_migraine_data.isnull().sum())

# Check for zero values
print("\nFiltered Migraine Zero Counts:\n")
zero_counts = (filtered_migraine_data == 0).sum()
print(zero_counts)


Filtered Migraine Missing Values:

measure     0
location    0
sex         0
age         0
cause       0
metric      0
year        0
val         0
upper       0
lower       0
dtype: int64

Filtered Migraine Zero Counts:

measure        0
location       0
sex            0
age            0
cause          0
metric         0
year           0
val         6100
upper       6100
lower       6100
dtype: int64


No missing values in this dataset.

Plan: Merge with combined and filtered weather data.

We need to address the zero values in the `val`, `upper`, and `lower` columns. We will investigate the distribution of these values to determine the best method for handling them.

In [16]:
# Print rows with zero values
zero_rows_any = filtered_migraine_data[(filtered_migraine_data == 0).any(axis=1)]
print(zero_rows_any)

           measure                    location     sex       age  \
7434    Prevalence  Taiwan (Province of China)  Female  <5 years   
7440    Prevalence  Taiwan (Province of China)  Female  <5 years   
7446    Prevalence  Taiwan (Province of China)  Female  <5 years   
7452    Prevalence  Taiwan (Province of China)  Female  <5 years   
7458    Prevalence  Taiwan (Province of China)  Female  <5 years   
...            ...                         ...     ...       ...   
365212  Prevalence                    Oklahoma    Male  <5 years   
365218  Prevalence                    Oklahoma    Male  <5 years   
365224  Prevalence                    Oklahoma    Male  <5 years   
365230  Prevalence                    Oklahoma    Male  <5 years   
365236  Prevalence                    Oklahoma    Male  <5 years   

                        cause  metric  year  val  upper  lower  
7434    Tension-type headache  Number  1990 0.00   0.00   0.00  
7440    Tension-type headache  Number  1991 0.00   0.

In [17]:
# Check the counts of the population's age groups
filtered_migraine_data['age'].value_counts()

age
<5 years       6100
15-19 years    5833
75-84 years    5405
20-24 years    5399
65-74 years    5154
25-29 years    5075
35-39 years    5008
40-44 years    4990
5-14 years     4984
30-34 years    4984
85+ years      4922
45-49 years    4790
60-64 years    4661
50-54 years    4645
55-59 years    4635
Name: count, dtype: int64

The number of zero values in the `val`, `upper`, and `lower` columns is 6,100. After further investigation, there are also 6,100 rows where the age of the population is <5 years old. Since this is perfectly reasonable explanation, we will remove these rows from the dataset and this analysis.

In [18]:
# Drop rows that meet the condition
filtered_migraine_data.drop(
    filtered_migraine_data.query("`age` == '<5 years' and `val` == 0").index, 
    inplace=True
)

In [19]:
zero_rows_any = filtered_migraine_data[(filtered_migraine_data == 0).any(axis=1)]
print(zero_rows_any)

Empty DataFrame
Columns: [measure, location, sex, age, cause, metric, year, val, upper, lower]
Index: []


In [20]:
filtered_migraine_data['age'].value_counts()

age
15-19 years    5030
20-24 years    4661
40-44 years    4599
75-84 years    4549
35-39 years    4518
30-34 years    4453
5-14 years     4404
25-29 years    4380
45-49 years    4327
65-74 years    4287
85+ years      4164
60-64 years    4085
50-54 years    4039
55-59 years    3948
Name: count, dtype: int64

In [21]:
filtered_migraine_data.shape

(61444, 10)

In [23]:
filtered_migraine_data

,measure,location,sex,age,cause,metric,year,val,upper,lower
3415,Prevalence,Taiwan (Province of China),Male,85+ years,Migraine,Number,2011,5249.38,6903.76,3990.16
3421,Prevalence,Taiwan (Province of China),Male,85+ years,Migraine,Number,2012,5618.06,7341.49,4266.90
3427,Prevalence,Taiwan (Province of China),Male,85+ years,Migraine,Number,2013,5999.05,7848.98,4565.28
3433,Prevalence,Taiwan (Province of China),Male,85+ years,Migraine,Number,2014,6424.09,8403.95,4870.36
3439,Prevalence,Taiwan (Province of China),Male,85+ years,Migraine,Number,2015,6721.02,8764.60,5091.28
...,...,...,...,...,...,...,...,...,...,...
376485,Prevalence,Texas,Female,60-64 years,Tension-type headache,Number,1994,136301.30,177640.61,99142.23
376490,Prevalence,Texas,Male,60-64 years,Tension-type headache,Number,1995,111548.80,148253.99,79929.00
376491,Prevalence,Texas,Female,60-64 years,Tension-type headache,Number,1995,136406.47,177886.24,99248.86
376496,Prevalence,Texas,Male,60-64 years,Tension-type headache,Number,1996,112390.22,149308.16,80564.24


### 4.3 Standardize Country and State Names across datasets

#### 4.3.1 DataFrame: `city_data`

##### 4.3.1.1 Country Names

In [24]:
# Import the function
from data_location_matcher import find_matching_and_non_matching

# Find matching and non-matching countries
city_data_matching_countries, city_data_non_matching_countries = find_matching_and_non_matching(city_data, 'country')

# View the non-matching countries
city_data_non_matching_countries

{'Bahamas',
 'Barbados',
 'Cabo Verde',
 "Cote d'Ivoire",
 'Czechia',
 'Eritrea',
 'Eswatini',
 'Gambia',
 'Guinea-Bissau',
 'Korea, North',
 'Korea, South',
 'Kosovo',
 'Micronesia',
 'Nauru',
 'Palau',
 'Palestine',
 'Panama',
 'Saint Lucia',
 'Saint Vincent and the Grenadines',
 'Sao Tome and Principe',
 'Timor-Leste',
 'Tonga',
 'Vatican City'}

In [25]:
city_data_country_replacement_dict = { 
    'Guinea Bissau': 'Guinea-Bissau',
    'Korea, North': 'North Korea',
    'Korea, South': 'South Korea',
    'Macau S.A.R': 'Macau',
    'Svalbard and Jan Mayen Islands': 'Svalbard and Jan Mayen',
    'São Tomé and Príncipe': 'Sao Tome and Principe',
    'The Bahamas': 'Bahamas',
    'The Gambia': 'Gambia',
    'United States': 'United States of America'
}

# Replace the country names in the city dataframe
city_data['country'].replace(city_data_country_replacement_dict, inplace=True)

city_data['country'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan',
       'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso',
       'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Cayman Islands', 'Central African Republic', 'Chad', 'Chile',
       'China', 'Christmas Island', 'Colombia', 'Comoros',
       'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Cook Islands',
       'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic',
       'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic',
       'East Timor', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Estonia', 'Ethiopia', 'Falkland Islands',
       'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia',

##### 4.3.1.2 State Names

In [26]:
# Find matching and non-matching states
city_data_matching_states, city_data_non_matching_states = find_matching_and_non_matching(city_data, 'state')

# View the non-matching states
city_data_non_matching_states

{'Louisiana', 'South Dakota'}

#### 4.3.2 DataFrame: `country_data`

##### 4.3.2.1 Country Names

In [27]:
# Find matching and non-matching countries
country_data_matching_countries, country_data_non_matching_countries = find_matching_and_non_matching(country_data, 'country')

# View the non-matching countries
country_data_non_matching_countries

{'Andorra',
 'Bahamas',
 'Barbados',
 'Cabo Verde',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 "Cote d'Ivoire",
 'Czechia',
 'Eritrea',
 'Eswatini',
 'Gambia',
 'Korea, North',
 'Korea, South',
 'Kosovo',
 'Micronesia',
 'Montenegro',
 'Myanmar',
 'Nauru',
 'Palau',
 'Palestine',
 'Panama',
 'Saint Lucia',
 'Saint Vincent and the Grenadines',
 'Sao Tome and Principe',
 'Timor-Leste',
 'Tonga',
 'United States of America',
 'Vatican City'}

In [28]:
country_data_country_replacement_dict = {
    'Democratic Republic of the Congo': 'Congo (Kinshasa)',
    'Republic of the Congo': 'Congo (Brazzaville)',
    'Korea, North': 'North Korea',
    'Korea, South': 'South Korea',
    'São Tomé and Príncipe': 'Sao Tome and Principe', 
    'The Bahamas': 'Bahamas',
    'The Gambia': 'Gambia',
    'United States': 'United States of America'
}

# Replace the country names in the country dataframe
country_data['country'].replace(country_data_country_replacement_dict, inplace=True)

country_data['country'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Angola',
       'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh',
       'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia',
       'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands',
       'Central African Republic', 'Chad', 'Chile', 'China',
       'Christmas Island', 'Colombia', 'Comoros', 'Cook Islands',
       'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic',
       'Congo (Kinshasa)', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'East Timor', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Estonia', 'Ethiopia',
       'Falkland Islands', 'Fiji', 'Finland', 'France', 'French Guiana',
       'French Polynesia', 'French Southern and Antarctic La

##### 4.3.2.2 State Names

No state names in this dataset.

#### 4.3.3 DataFrame: `migraine_data`

##### 4.3.3.1 Country Names

In [29]:
filtered_migraine_data.head()

,measure,location,sex,age,cause,metric,year,val,upper,lower
3415,Prevalence,Taiwan (Province of China),Male,85+ years,Migraine,Number,2011,5249.38,6903.76,3990.16
3421,Prevalence,Taiwan (Province of China),Male,85+ years,Migraine,Number,2012,5618.06,7341.49,4266.90
3427,Prevalence,Taiwan (Province of China),Male,85+ years,Migraine,Number,2013,5999.05,7848.98,4565.28
3433,Prevalence,Taiwan (Province of China),Male,85+ years,Migraine,Number,2014,6424.09,8403.95,4870.36
3439,Prevalence,Taiwan (Province of China),Male,85+ years,Migraine,Number,2015,6721.02,8764.60,5091.28


In [30]:
from data_location_matcher import COUNTRIES, US_STATES

migraine_data_countries_and_states = filtered_migraine_data['location'].unique()

migraine_data_countries_and_states

array(['Taiwan (Province of China)', 'Marshall Islands', 'Viet Nam',
       'Philippines', 'Samoa', 'Albania', 'Timor-Leste', 'Tajikistan',
       'Myanmar', 'Georgia', "Democratic People's Republic of Korea",
       "Lao People's Democratic Republic", 'Azerbaijan', 'Thailand',
       'Mongolia', 'China', 'Fiji', 'Maldives', 'Papua New Guinea',
       'Vanuatu', 'Uzbekistan', 'Micronesia (Federated States of)',
       'Malaysia', 'Armenia', 'Cambodia', 'Tonga', 'Turkmenistan',
       'Montenegro', 'Sri Lanka', 'Solomon Islands', 'Slovakia',
       'Croatia', 'Bulgaria', 'Australia', 'Brunei Darussalam',
       'North Macedonia', 'Greece', 'Hungary', 'Denmark', 'Serbia',
       'Lithuania', 'Andorra', 'Republic of Moldova',
       'Bosnia and Herzegovina', 'Germany', 'Singapore', 'Romania',
       'Cyprus', 'Poland', 'Czechia', 'Latvia', 'Norway', 'Israel',
       'Switzerland', 'Ukraine', 'Belgium', 'Republic of Korea', 'France',
       'Ireland', 'Barbados', 'Chile', 'Sweden', 'Japan'

In [31]:
migraine_data_location_replacement_dict = {
    'Taiwan (Province of China)': 'Taiwan',
    'Viet Nam': 'Vietnam',
    "Democratic People's Republic of Korea": 'North Korea',
    "Lao People's Democratic Republic": 'Laos',
    'Democratic Republic of the Congo': 'Congo (Kinshasa)',
    'Micronesia (Federated States of)': 'Micronesia',
    'North Macedonia': 'Macedonia',
    'Brunei Darussalam': 'Brunei',
    'Republic of Korea': 'South Korea',
    'Bolivia (Plurinational State of)': 'Bolivia',
    'Venezuela (Bolivarian Republic of)': 'Venezuela',
    'Iran (Islamic Republic of)': 'Iran',
    'United Republic of Tanzania': 'Tanzania',    
    'Republic of the Congo': 'Congo (Brazzaville)',
    'Republic of Moldova': 'Moldova',
    'Korea, North': 'North Korea',
    'Korea, South': 'South Korea',
    'São Tomé and Príncipe': 'Sao Tome and Principe', 
    'The Bahamas': 'Bahamas',
    'The Gambia': 'Gambia',
    'United States': 'United States of America'
}

# Replace the country names in the country dataframe
filtered_migraine_data['location'].replace(migraine_data_location_replacement_dict, inplace=True)

filtered_migraine_data['location'].unique()

array(['Taiwan', 'Marshall Islands', 'Vietnam', 'Philippines', 'Samoa',
       'Albania', 'Timor-Leste', 'Tajikistan', 'Myanmar', 'Georgia',
       'North Korea', 'Laos', 'Azerbaijan', 'Thailand', 'Mongolia',
       'China', 'Fiji', 'Maldives', 'Papua New Guinea', 'Vanuatu',
       'Uzbekistan', 'Micronesia', 'Malaysia', 'Armenia', 'Cambodia',
       'Tonga', 'Turkmenistan', 'Montenegro', 'Sri Lanka',
       'Solomon Islands', 'Slovakia', 'Croatia', 'Bulgaria', 'Australia',
       'Brunei', 'Macedonia', 'Greece', 'Hungary', 'Denmark', 'Serbia',
       'Lithuania', 'Andorra', 'Moldova', 'Bosnia and Herzegovina',
       'Germany', 'Singapore', 'Romania', 'Cyprus', 'Poland', 'Czechia',
       'Latvia', 'Norway', 'Israel', 'Switzerland', 'Ukraine', 'Belgium',
       'South Korea', 'France', 'Ireland', 'Barbados', 'Chile', 'Sweden',
       'Japan', 'Finland', 'Iceland', 'Netherlands', 'Dominica',
       'United States of America', 'Haiti', 'Bahamas', 'Malta',
       'Luxembourg', 'Argentina

In [33]:
# Convert original list and the list of U.S. states to sets
migraine_data_countries_and_states = filtered_migraine_data["location"].unique()

set_migraine_countries_and_states = set(migraine_data_countries_and_states)
set_US_states = set(US_STATES)
set_countries = set(COUNTRIES)

# Create a new list excluding the U.S. states
migrained_filtered_countries_list = [
    item for item in set_migraine_countries_and_states if item not in set_US_states
]

# View the list
migrained_filtered_countries_list

['Mongolia',
 'Bhutan',
 'South Sudan',
 'Tajikistan',
 'Bahamas',
 'Ghana',
 'Denmark',
 'Solomon Islands',
 'Bahrain',
 'Mali',
 'Congo (Kinshasa)',
 'Palestine',
 'Montenegro',
 'Ukraine',
 'Cyprus',
 'Singapore',
 'Dominican Republic',
 'Albania',
 'Niger',
 'Lithuania',
 'Israel',
 'Barbados',
 'Puerto Rico',
 'Equatorial Guinea',
 'Dominica',
 'France',
 'Armenia',
 'Guatemala',
 'Bosnia and Herzegovina',
 'Greece',
 'Ireland',
 'China',
 'Somalia',
 'Gabon',
 'Togo',
 'Saudi Arabia',
 "Côte d'Ivoire",
 'Saint Kitts and Nevis',
 'Germany',
 'Lebanon',
 'Bolivia',
 'Chile',
 'Nigeria',
 'Argentina',
 'Ethiopia',
 'Papua New Guinea',
 'Sao Tome and Principe',
 'Andorra',
 'Honduras',
 'Finland',
 'Tanzania',
 'Senegal',
 'Venezuela',
 'Comoros',
 'Liberia',
 'Tokelau',
 'Bangladesh',
 'Mozambique',
 'Guam',
 'Trinidad and Tobago',
 'Jordan',
 'Zimbabwe',
 'Cook Islands',
 'Moldova',
 'Cameroon',
 'Maldives',
 'Poland',
 'Burundi',
 'Turkmenistan',
 'Uruguay',
 'Central African Repu

##### 4.3.3.2 State Names

In [34]:
# Convert the list of U.S. states to sets
set_US_states = set(US_STATES)

# Create a new list including the U.S. states
migrained_filtered_states_list = [item for item in set_migraine_countries_and_states if item in set_US_states]

# View the list
migrained_filtered_states_list

['Connecticut',
 'Wisconsin',
 'Indiana',
 'Utah',
 'Arkansas',
 'Maryland',
 'New Jersey',
 'Illinois',
 'Vermont',
 'District of Columbia',
 'Kansas',
 'Wyoming',
 'Arizona',
 'Missouri',
 'Florida',
 'Oregon',
 'Alabama',
 'Mississippi',
 'Montana',
 'Colorado',
 'Virginia',
 'Nebraska',
 'Maine',
 'Michigan',
 'Georgia',
 'Delaware',
 'New York',
 'New Mexico',
 'Ohio',
 'South Carolina',
 'Iowa',
 'North Dakota',
 'Massachusetts',
 'South Dakota',
 'New Hampshire',
 'Rhode Island',
 'Tennessee',
 'Hawaii',
 'West Virginia',
 'Louisiana',
 'Washington',
 'Oklahoma',
 'North Carolina',
 'Alaska',
 'Texas']

## 5. Data Integration

### 5.1 Overview

Briefly introduce the goal of data integration in the context of this project. Provide a high-level view of the datasets that will be integrated.

### 5.2 Data Sources

#### 5.2.1 Weather Data

The weather data provides context regarding sea level pressure, sunshine, temperature, and precipitation for each city. This data is relevant because it provides information about the weather conditions that may be associated with migraine prevalence. The country data will be combined with the city data to provide additional information about each city, such as the country, region, and continent. The combined city and country data will then be combined with the weather data to provide additional information.

The daily weather data source file is quite large and is provided in a .parquet format for low memory consumption and data type preservation. 

##### 5.2.1.1 Cities DataFrame

| Attribute            | Description                                        |
|----------------------|----------------------------------------------------|
| **Data Source Name** | cities.csv                                         |
| **Data Source Format** | CSV (comma-separated values)                       |
| **Data Source Desc** | Individual cities and weather stations around the world |
| **Data Source Size** | 84.1 KB                                             |
|                      | 1,245 rows                                          |
|                      | 8 columns                                           |
| **Data Source Limits** | None                                              |
| **Data Source Usability** | 10.00                                          |

**Data Source Columns**

| Column Name  | Description                               |
|--------------|-------------------------------------------|
| `station_id` | Unique ID for the weather station.        |
| `city_name`  | Name of the city.                         |
| `country`    | The country where the city is located.    |
| `state`      | The state or province within the country. |
| `iso2`       | The two-letter country code.              |
| `iso3`       | The three-letter country code.            |
| `latitude`   | Latitude coordinate of the city.          |
| `longitude`  | Longitude coordinate of the city.         |


##### 5.2.1.2 Countries DataFrame

| Attribute            | Description                                        |
|----------------------|----------------------------------------------------|
| **Data Source Name** | countries.csv                                         |
| **Data Source Format** | CSV (comma-separated values)                       |
| **Data Source Desc** | Individual country geographic and demographic characteristics |
| **Data Source Size** | 20.6 KB                        |
|                      | 214 rows                                    |
|                      | 11 columns                                       |
| **Data Source Limits** | None                                              |
| **Data Source Usability** | 10.00                                           |

**Data Source Columns**

| Column Name  | Description                                               |
|--------------|-----------------------------------------------------------|
| `iso3`       | The three-letter code representing the country.           |
| `country`    | The English name of the country.                          |
| `native_name`| The native name of the country.                           |
| `iso2`       | The two-letter code representing the country.             |
| `population` | The population of the country.                            |
| `area`       | The total land area of the country in square kilometers.  |
| `capital`    | The name of the capital city.                             |
| `capital_lat`| The latitude coordinate of the capital city.              |
| `capital_lng`| The longitude coordinate of the capital city.             |
| `region`     | The specific region within the continent where the country is located. |
| `continent`  | The continent to which the country belongs.               |

##### 5.2.1.3 Daily Weather DataFrame

| Attribute            | Description                                        |
|----------------------|----------------------------------------------------|
| **Data Source Name** | daily_weather.parquet                              |
| **Data Source Format** | .parquet (compressed, maintains original data types, efficient)|
| **Data Source Desc** | Daily weather data                            |
| **Data Source Size** | 233 MB                                        |
|                      | 27,635,763 rows                               |
|                      | 14 columns                                    |
| **Data Source Limits** | None                                              |
| **Data Source Usability** | 10.00                                           |

**Data Source Columns**

| Column Name            | Description                                       |
|------------------------|---------------------------------------------------|
| `station_id`           | Unique ID for the weather station.                |
| `city_name`            | Name of the city where the station is located.    |
| `date`                 | Date of the weather record.                       |
| `season`               | Season corresponding to the date (e.g., summer, winter).|
| `avg_temp_c`           | Average temperature in Celsius.                   |
| `min_temp_c`           | Minimum temperature in Celsius.                   |
| `max_temp_c`           | Maximum temperature in Celsius.                   |
| `precipitation_mm`     | Precipitation in millimeters.                     |
| `snow_depth_mm`        | Snow depth in millimeters.                        |
| `avg_wind_dir_deg`     | Average wind direction in degrees.                |
| `avg_wind_speed_kmh`   | Average wind speed in kilometers per hour.        |
| `peak_wind_gust_kmh`   | Peak wind gust in kilometers per hour.            |
| `avg_sea_level_pres_hpa`| Average sea-level pressure in hectopascals.      |
| `sunshine_total_min`   | Total sunshine duration in minutes.               |

#### 5.2.2 Migraine Data

The migraine data provides information about the prevalence of migraine in different countries. This data is relevant because it provides information about the prevalence of migraine by gender, age, year, and location. This data will be combined with the weather data to determine if there is a relationship between weather and migraine prevalence.

| Attribute            | Description                                        |
|----------------------|----------------------------------------------------|
| **Data Source Name** | IHME-GBD_2019_DATA-2c1d3941-1.csv                  |
|                      | IHME-GBD_2019_DATA-2c1d3941-2.csv                  |
|                      | IHME-GBD_2019_DATA-2c1d3941-3.csv                  |
| **Data Source Format** | CSV (comma-separated values)                     |
| **Data Source Desc** | All GBD causes, risks, impairments, etiologies, and injuries by nature |
| **Data Source Size** | 158 MB                                              |
|                      | 1,377,000 rows                                      |
|                      | 10 columns                                          |
| **Data Source Limits** | None                                              |
| **Data Source Usability** | 10.00                                          |

**Data Source Columns**

| Column Name    | Description                                          |
|----------------|------------------------------------------------------|
| `measure` | The name of measure.                                      |
| `location`| The name of each location.                                |
| `sex`     | The name of each sex choice.                              |
| `age`     | The name of each age group.                               |
| `cause`   | The name of each cause.                                   |
| `metric`  | The name of each metric/unit.                             |
| `year`    | The annual results for all measures.                      |
| `val`     | The value of each metric/unit.                            |
| `upper`   | The 95% Uncertainty Interval - Upper Bound value.         |
| `lower`   | The 95% Uncertainty Interval - Lower Bound value.         |

### 5.3 Preliminary Steps

**Overview**

In this section, the focus is on preparing the dataset for further analysis and exploration. The steps include merging multiple data sources, filtering the data based on specific criteria, cleaning the data by dropping unnecessary columns and rows, and conducting a preliminary analysis through correlation metrics. Each of these steps is essential for ensuring the data's integrity, usability, and relevance to the study objectives.

**5.3.1 Data Merging**

The first step involves merging the city and country datasets using a common identifier. This integration provides a comprehensive view that combines geographical and political attributes. Following that, the weather dataset is integrated with the already combined city-country data. The resulting dataset offers a rich context, incorporating both geographical information and meteorological variables.

**5.3.2 Data Filtering**

The dataset is filtered to only include records pertaining to US cities, thereby narrowing the scope for more targeted analysis. Further filtering is done to include only specific years, enhancing the dataset's relevance to the study period.

**5.3.3 Data Cleaning**

Columns that do not contribute to the analysis or contain redundant information are dropped to simplify the dataset. Rows with missing or irrelevant data are removed to improve the dataset's quality and consistency. Duplicate rows, if any, are identified and removed to ensure each record in the dataset is unique.

**5.3.4 Preliminary Analysis**

A correlation analysis is conducted on specific weather attributes like temperature, precipitation, and wind speed to identify any significant relationships among them.

---

Throughout these steps, the data are continuously inspected to understand their structures, types, and quality. Various data profiling techniques are employed, such as examining data distributions, checking for missing values, and assessing data types, to ensure that the dataset meets the quality and integrity requirements for downstream analysis.

#### 5.3.1 Data Merging

##### 5.3.1.1 Merge `city_data` and `country_data`

Join the countries and cities tables on the `country`, `iso2`, and `iso3` columns to give more context to the weather data.

In [35]:
# Code for joining countries and cities
city_country = city_data.merge(country_data, 
                               how='left', 
                               left_on=['country', 'iso2', 'iso3'], 
                               right_on=['country', 'iso2', 'iso3']
                               )

# Review the shape of the new dataframe
city_country

,station_id,city_name,country,state,iso2,iso3,latitude,longitude,population,area
0,41515,Asadabad,Afghanistan,Kunar,AF,AFG,34.87,71.15,26023100.00,652230.00
1,38954,Fayzabad,Afghanistan,Badakhshan,AF,AFG,37.13,70.58,26023100.00,652230.00
2,41560,Jalalabad,Afghanistan,Nangarhar,AF,AFG,34.44,70.44,26023100.00,652230.00
3,38947,Kunduz,Afghanistan,Kunduz,AF,AFG,36.73,68.87,26023100.00,652230.00
4,38987,Qala i Naw,Afghanistan,Badghis,AF,AFG,34.98,63.13,26023100.00,652230.00
...,...,...,...,...,...,...,...,...,...,...
1240,67475,Kasama,Zambia,Northern,ZM,ZMB,-10.20,31.18,15023315.00,752612.00
1241,68030,Livingstone,Zambia,Southern,ZM,ZMB,-17.86,25.86,15023315.00,752612.00
1242,67633,Mongu,Zambia,Western,ZM,ZMB,-15.28,23.12,15023315.00,752612.00
1243,67775,Harare,Zimbabwe,Harare,ZW,ZWE,-17.82,31.04,13061239.00,390757.00


##### 5.3.1.2 Merge `weather_data` and `city_country`

Join the weather data with the combined countries and cities tables on the `station_id` and `city_name` columns.

In [36]:
# Review the shape of the weather dataframe
print(f"Weather Data: {weather_data.shape}")

# Review the shape of the city-country dataframe
print(f"City-Country Data: {city_country.shape}")

Weather Data: (27635763, 14)
City-Country Data: (1245, 10)


In [37]:
weather_data.head(1)

,station_id,city_name,date,season,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,snow_depth_mm,avg_wind_dir_deg,avg_wind_speed_kmh,peak_wind_gust_kmh,avg_sea_level_pres_hpa,sunshine_total_min
0,41515,Asadabad,1957-07-01,Summer,27.00,21.10,35.60,0.00,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# Combine city/country with daily weather data
combined_weather = weather_data.merge(city_country, 
                                      how='left', 
                                      left_on=['station_id', 'city_name'], 
                                      right_on=['station_id', 'city_name']
)

# Review the shape of the new dataframe
print(f"Combined Weather Data: {combined_weather.shape}")

Combined Weather Data: (27678323, 22)


In [39]:
combined_weather.head(1)

,station_id,city_name,date,season,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,snow_depth_mm,avg_wind_dir_deg,avg_wind_speed_kmh,peak_wind_gust_kmh,avg_sea_level_pres_hpa,sunshine_total_min,country,state,iso2,iso3,latitude,longitude,population,area
0,41515,Asadabad,1957-07-01,Summer,27.00,21.10,35.60,0.00,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,Kunar,AF,AFG,34.87,71.15,26023100.00,652230.00


In [40]:
combined_weather['country'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Anguilla', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan',
       'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso',
       'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde',
       'Cayman Islands', 'Central African Republic', 'Chad', 'Chile',
       'China', 'Christmas Island', 'Colombia', 'Comoros',
       'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Cook Islands',
       'Costa Rica', 'Croatia', 'Cuba', 'Cyprus', 'Czech Republic',
       'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic',
       'East Timor', 'Ecuador', 'Egypt', 'El Salvador',
       'Equatorial Guinea', 'Estonia', 'Ethiopia', 'Falkland Islands',
       'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia',

In [41]:
# View the states where country is 'United States of America'
combined_weather[combined_weather['country'] == 'United States of America']['state'].unique()

array(['New York', 'Maryland', 'Georgia', 'Maine', 'Texas',
       'North Dakota', 'Idaho', 'Massachusetts', 'Nevada',
       'West Virginia', 'Wyoming', 'South Carolina', 'Ohio',
       'New Hampshire', 'Colorado', 'Iowa', 'Delaware', 'Kentucky',
       'Pennsylvania', 'Connecticut', 'Montana', 'Hawaii', 'Indiana',
       'Mississippi', 'Missouri', 'Alaska', 'Michigan', 'Nebraska',
       'Arkansas', 'Wisconsin', 'Alabama', 'Vermont', 'Tennessee',
       'Oklahoma', 'Washington', 'Arizona', 'Rhode Island',
       'North Carolina', 'Virginia', 'California', 'Minnesota', 'Oregon',
       'Utah', 'New Mexico', 'Illinois', 'Florida', 'Kansas',
       'New Jersey', 'District of Columbia'], dtype=object)

In [42]:
combined_weather.isnull().sum()

station_id                       0
city_name                    13993
date                             0
season                           0
avg_temp_c                 6230907
min_temp_c                 5723019
max_temp_c                 5544972
precipitation_mm           6664436
snow_depth_mm             24251175
avg_wind_dir_deg          24212511
avg_wind_speed_kmh        22372707
peak_wind_gust_kmh        26556837
avg_sea_level_pres_hpa    23648694
sunshine_total_min        26656862
country                          0
state                       546360
iso2                        130781
iso3                             0
latitude                         0
longitude                        0
population                  511609
area                        214223
dtype: int64

##### 5.3.1.3 Add `year` column to `combined_weather` and Filter to 2010-2019

Given that the migraine data is annual, we need to add a 'year' column to the weather data and filter it by year.

In [43]:
# confirmed that date column is in datetime format
combined_weather.dtypes

station_id                        object
city_name                         object
date                      datetime64[ns]
season                          category
avg_temp_c                       float64
min_temp_c                       float64
max_temp_c                       float64
precipitation_mm                 float64
snow_depth_mm                    float64
avg_wind_dir_deg                 float64
avg_wind_speed_kmh               float64
peak_wind_gust_kmh               float64
avg_sea_level_pres_hpa           float64
sunshine_total_min               float64
country                           object
state                             object
iso2                              object
iso3                              object
latitude                         float64
longitude                        float64
population                       float64
area                             float64
dtype: object

In [44]:
# Make a copy of the dataframe
combined_weather = combined_weather.copy()

# Create a new column for the year
combined_weather['year'] = combined_weather['date'].dt.year

combined_weather

,station_id,city_name,date,season,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,snow_depth_mm,avg_wind_dir_deg,avg_wind_speed_kmh,peak_wind_gust_kmh,avg_sea_level_pres_hpa,sunshine_total_min,country,state,iso2,iso3,latitude,longitude,population,area,year
0,41515,Asadabad,1957-07-01,Summer,27.00,21.10,35.60,0.00,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,Kunar,AF,AFG,34.87,71.15,26023100.00,652230.00,1957
1,41515,Asadabad,1957-07-02,Summer,22.80,18.90,32.20,0.00,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,Kunar,AF,AFG,34.87,71.15,26023100.00,652230.00,1957
2,41515,Asadabad,1957-07-03,Summer,24.30,16.70,35.60,1.00,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,Kunar,AF,AFG,34.87,71.15,26023100.00,652230.00,1957
3,41515,Asadabad,1957-07-04,Summer,26.60,16.10,37.80,4.10,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,Kunar,AF,AFG,34.87,71.15,26023100.00,652230.00,1957
4,41515,Asadabad,1957-07-05,Summer,30.80,20.00,41.70,0.00,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,Kunar,AF,AFG,34.87,71.15,26023100.00,652230.00,1957


In [45]:
combined_weather['year'].unique()

array([1957, 1958, 1961, 1962, 1973, 1974, 1975, 1976, 1977, 1978, 1979,
       1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990,
       1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023,
       1898, 1899, 1900, 1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908,
       1909, 1910, 1911, 1912, 1913, 1914, 1915, 1916, 1917, 1924, 1925,
       1926, 1927, 1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937,
       1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948,
       1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1959, 1960, 1963,
       1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1918, 1921,
       1922, 1923, 1928, 1880, 1881, 1882, 1883, 1884, 1885, 1886, 1887,
       1888, 1889, 1890, 1891, 1892, 1893, 1897, 1919, 1920, 1895, 1896,
       1879, 1894, 1750, 1877, 1878, 1876, 1874, 18

In [46]:
combined_weather['year'].describe()

count   27678323.00
mean        1982.43
std           30.36
min         1750.00
25%         1965.00
50%         1988.00
75%         2007.00
max         2023.00
Name: year, dtype: float64

Earliest year is 1750, latest year is 2023. We will filter the weather data to only include years 1990-2019 to match the migraine data's date range.

In [47]:
combined_weather.shape

(27678323, 23)

In [48]:
combined_weather.isnull().sum()

station_id                       0
city_name                    13993
date                             0
season                           0
avg_temp_c                 6230907
min_temp_c                 5723019
max_temp_c                 5544972
precipitation_mm           6664436
snow_depth_mm             24251175
avg_wind_dir_deg          24212511
avg_wind_speed_kmh        22372707
peak_wind_gust_kmh        26556837
avg_sea_level_pres_hpa    23648694
sunshine_total_min        26656862
country                          0
state                       546360
iso2                        130781
iso3                             0
latitude                         0
longitude                        0
population                  511609
area                        214223
year                             0
dtype: int64

In [49]:
# Filter the data to only include the years 1990-2019
year_filter = (combined_weather['year'] >= 1990) & (combined_weather['year'] <= 2019)
combined_weather = combined_weather[year_filter]

# Review the shape of the new dataframe
combined_weather.shape

(11551553, 23)

In [50]:
combined_weather['year'].unique()

array([1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
       2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019])

In [51]:
combined_weather.isnull().sum()

station_id                       0
city_name                    10950
date                             0
season                           0
avg_temp_c                  407052
min_temp_c                 2183181
max_temp_c                 1875695
precipitation_mm           3885419
snow_depth_mm             10089601
avg_wind_dir_deg          10079608
avg_wind_speed_kmh         8620070
peak_wind_gust_kmh        11186025
avg_sea_level_pres_hpa     9821732
sunshine_total_min        11174322
country                          0
state                       283904
iso2                         41868
iso3                             0
latitude                         0
longitude                        0
population                  238872
area                        116713
year                             0
dtype: int64

In [52]:
combined_weather.shape

(11551553, 23)

In [65]:
combined_weather.head(1)

,station_id,city_name,date,season,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,snow_depth_mm,avg_wind_dir_deg,avg_wind_speed_kmh,peak_wind_gust_kmh,avg_sea_level_pres_hpa,sunshine_total_min,country,state,iso2,iso3,latitude,longitude,population,area,year
2080,41515,Asadabad,1990-01-21,Winter,8.70,3.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghanistan,Kunar,AF,AFG,34.87,71.15,26023100.00,652230.00,1990


In [75]:
# Group by 'year' and 'state', then aggregate the numerical columns
annual_weather_by_stateCountry = (
    combined_weather.groupby(["year", "country", "state"])
    .agg(
        {
            "avg_temp_c": "mean",
            "min_temp_c": "mean",
            "max_temp_c": "mean",
            "precipitation_mm": "sum",
            "snow_depth_mm": "sum",
            "avg_wind_dir_deg": "mean",
            "avg_wind_speed_kmh": "mean",
            "peak_wind_gust_kmh": "mean",
            "avg_sea_level_pres_hpa": "mean",
            "sunshine_total_min": "mean",
            "population": "mean",
            "area": "mean",
            "latitude": "first",  # Assuming all latitudes are the same for a given year and state
            "longitude": "first",  # Assuming all longitudes are the same for a given year and state
        }
    )
    .reset_index()
)

# Review the shape of the new dataframe
annual_weather_by_stateCountry.shape

(7736, 17)

In [76]:
annual_weather_by_stateCountry

,year,country,state,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,snow_depth_mm,avg_wind_dir_deg,avg_wind_speed_kmh,peak_wind_gust_kmh,avg_sea_level_pres_hpa,sunshine_total_min,population,area,latitude,longitude
0,1990,Afghanistan,None,14.49,7.76,21.80,1016.00,1303.00,NaN,NaN,NaN,NaN,NaN,26023100.00,652230.00,34.87,71.15
1,1990,Albania,None,13.05,7.09,18.90,483.60,7710.00,NaN,NaN,NaN,NaN,NaN,2895947.00,28748.00,41.68,20.43
2,1990,Algeria,None,18.68,13.30,24.49,5038.50,319.00,NaN,10.97,NaN,NaN,NaN,38700000.00,2381741.00,27.87,-0.29
3,1990,American Samoa,None,27.30,24.86,29.66,2676.60,0.00,117.32,20.18,43.30,1010.80,348.36,55519.00,199.00,-14.28,-170.71
4,1990,Andorra,None,12.61,7.18,19.39,855.20,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.50,1.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7731,2019,Wallis and Futuna,None,27.72,25.23,30.67,3909.10,0.00,136.77,11.68,NaN,1009.65,NaN,13135.00,142.00,-13.28,-176.17
7732,2019,Western Sahara,None,20.89,18.16,24.16,24.60,0.00,90.51,22.11,NaN,1018.61,NaN,586000.00,266000.00,27.15,-13.20
7733,2019,Yemen,None,31.13,27.17,34.95,71.10,0.00,204.59,11.74,NaN,1008.90,NaN,25956000.00,527968.00,15.35,44.21
7734,2019,Zambia,None,25.57,17.32,31.00,83.50,0.00,NaN,NaN,NaN,NaN,NaN,15023315.00,752612.00,-17.86,25.86


In [77]:
annual_weather_by_stateCountry.isnull().sum()

year                         0
country                      0
state                        0
avg_temp_c                 218
min_temp_c                  98
max_temp_c                  99
precipitation_mm             0
snow_depth_mm                0
avg_wind_dir_deg          2984
avg_wind_speed_kmh        2716
peak_wind_gust_kmh        7161
avg_sea_level_pres_hpa    4366
sunshine_total_min        6848
population                 169
area                       245
latitude                     0
longitude                    0
dtype: int64

##### 5.3.1.4 Merge `filtered_migraine_data` and `combined_weather_data`

Before we can merge the migraine data with the weather data, we need to split the standardized `location` column into `country` and `state` columns. We have already confirmed that the `city_country` dataframe contains both a country and state column so we will use this dataframe to populate the `country` and `state` columns in the migraine data. We will then drop the `location` column from the migraine data. Finally, we will merge the migraine data with the weather data on the `country`, `state`, and `year` columns.

In [83]:
filtered_migraine_data.head(1)

,measure,location,sex,age,cause,metric,year,val,upper,lower
3415,Prevalence,Taiwan,Male,85+ years,Migraine,Number,2011,5249.38,6903.76,3990.16


In [84]:
filtered_migraine_data['location'].unique()

array(['Taiwan', 'Marshall Islands', 'Vietnam', 'Philippines', 'Samoa',
       'Albania', 'Timor-Leste', 'Tajikistan', 'Myanmar', 'Georgia',
       'North Korea', 'Laos', 'Azerbaijan', 'Thailand', 'Mongolia',
       'China', 'Fiji', 'Maldives', 'Papua New Guinea', 'Vanuatu',
       'Uzbekistan', 'Micronesia', 'Malaysia', 'Armenia', 'Cambodia',
       'Tonga', 'Turkmenistan', 'Montenegro', 'Sri Lanka',
       'Solomon Islands', 'Slovakia', 'Croatia', 'Bulgaria', 'Australia',
       'Brunei', 'Macedonia', 'Greece', 'Hungary', 'Denmark', 'Serbia',
       'Lithuania', 'Andorra', 'Moldova', 'Bosnia and Herzegovina',
       'Germany', 'Singapore', 'Romania', 'Cyprus', 'Poland', 'Czechia',
       'Latvia', 'Norway', 'Israel', 'Switzerland', 'Ukraine', 'Belgium',
       'South Korea', 'France', 'Ireland', 'Barbados', 'Chile', 'Sweden',
       'Japan', 'Finland', 'Iceland', 'Netherlands', 'Dominica',
       'United States of America', 'Haiti', 'Bahamas', 'Malta',
       'Luxembourg', 'Argentina

In [85]:
# import the assign_country function
from cleanup_location_migraine import assign_country

# Assign country to migraine data
filtered_migraine_data['country'] = filtered_migraine_data['location'].apply(assign_country, args=(city_country,))

filtered_migraine_data.head(1)

,measure,location,sex,age,cause,metric,year,val,upper,lower,country
3415,Prevalence,Taiwan,Male,85+ years,Migraine,Number,2011,5249.38,6903.76,3990.16,Taiwan


In [90]:
# Check state values in filtered_migraine_data dataframe, if not US state, then replace with 'None'
filtered_migraine_data["state"] = filtered_migraine_data["location"].apply(
    lambda x: x if x in US_STATES else "None"
)

filtered_migraine_data["state"].unique()

array(['None', 'Georgia', 'Arizona', 'Hawaii', 'District of Columbia',
       'Michigan', 'Colorado', 'Kansas', 'Indiana', 'Missouri', 'Iowa',
       'Florida', 'Mississippi', 'Alaska', 'Massachusetts', 'Nebraska',
       'Illinois', 'Delaware', 'Louisiana', 'Maryland', 'Alabama',
       'Connecticut', 'Arkansas', 'New Hampshire', 'Montana', 'Maine',
       'New Mexico', 'West Virginia', 'Oregon', 'Tennessee', 'Wyoming',
       'Rhode Island', 'Vermont', 'South Dakota', 'South Carolina',
       'North Dakota', 'Virginia', 'New Jersey', 'Utah', 'Texas', 'Ohio',
       'North Carolina', 'Wisconsin', 'Washington', 'Oklahoma',
       'New York'], dtype=object)

In [91]:
filtered_migraine_data.isnull().sum()

measure     0
location    0
sex         0
age         0
cause       0
metric      0
year        0
val         0
upper       0
lower       0
country     0
state       0
dtype: int64

In [92]:
filtered_migraine_data.shape

(61444, 12)

In [93]:
filtered_migraine_data

,measure,location,sex,age,cause,metric,year,val,upper,lower,country,state
3415,Prevalence,Taiwan,Male,85+ years,Migraine,Number,2011,5249.38,6903.76,3990.16,Taiwan,None
3421,Prevalence,Taiwan,Male,85+ years,Migraine,Number,2012,5618.06,7341.49,4266.90,Taiwan,None
3427,Prevalence,Taiwan,Male,85+ years,Migraine,Number,2013,5999.05,7848.98,4565.28,Taiwan,None
3433,Prevalence,Taiwan,Male,85+ years,Migraine,Number,2014,6424.09,8403.95,4870.36,Taiwan,None
3439,Prevalence,Taiwan,Male,85+ years,Migraine,Number,2015,6721.02,8764.60,5091.28,Taiwan,None
...,...,...,...,...,...,...,...,...,...,...,...,...
376485,Prevalence,Texas,Female,60-64 years,Tension-type headache,Number,1994,136301.30,177640.61,99142.23,United States of America,Texas
376490,Prevalence,Texas,Male,60-64 years,Tension-type headache,Number,1995,111548.80,148253.99,79929.00,United States of America,Texas
376491,Prevalence,Texas,Female,60-64 years,Tension-type headache,Number,1995,136406.47,177886.24,99248.86,United States of America,Texas
376496,Prevalence,Texas,Male,60-64 years,Tension-type headache,Number,1996,112390.22,149308.16,80564.24,United States of America,Texas


In [94]:
annual_weather_by_stateCountry

,year,country,state,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,snow_depth_mm,avg_wind_dir_deg,avg_wind_speed_kmh,peak_wind_gust_kmh,avg_sea_level_pres_hpa,sunshine_total_min,population,area,latitude,longitude
0,1990,Afghanistan,None,14.49,7.76,21.80,1016.00,1303.00,NaN,NaN,NaN,NaN,NaN,26023100.00,652230.00,34.87,71.15
1,1990,Albania,None,13.05,7.09,18.90,483.60,7710.00,NaN,NaN,NaN,NaN,NaN,2895947.00,28748.00,41.68,20.43
2,1990,Algeria,None,18.68,13.30,24.49,5038.50,319.00,NaN,10.97,NaN,NaN,NaN,38700000.00,2381741.00,27.87,-0.29
3,1990,American Samoa,None,27.30,24.86,29.66,2676.60,0.00,117.32,20.18,43.30,1010.80,348.36,55519.00,199.00,-14.28,-170.71
4,1990,Andorra,None,12.61,7.18,19.39,855.20,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.50,1.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7731,2019,Wallis and Futuna,None,27.72,25.23,30.67,3909.10,0.00,136.77,11.68,NaN,1009.65,NaN,13135.00,142.00,-13.28,-176.17
7732,2019,Western Sahara,None,20.89,18.16,24.16,24.60,0.00,90.51,22.11,NaN,1018.61,NaN,586000.00,266000.00,27.15,-13.20
7733,2019,Yemen,None,31.13,27.17,34.95,71.10,0.00,204.59,11.74,NaN,1008.90,NaN,25956000.00,527968.00,15.35,44.21
7734,2019,Zambia,None,25.57,17.32,31.00,83.50,0.00,NaN,NaN,NaN,NaN,NaN,15023315.00,752612.00,-17.86,25.86


In [95]:
# Combine filtered migraine data with combined weather data
weather_migraine = filtered_migraine_data.merge(annual_weather_by_stateCountry, 
                                                  how="left", 
                                                  left_on=["year", "country", "state"], 
                                                  right_on=["year", "country", "state"]
                                                  )

# Review the shape of the new dataframe
print(f"Migraine Data and Combined Weather: {weather_migraine.shape}")

weather_migraine.head(1)

Migraine Data and Combined Weather: (61444, 26)


,measure,location,sex,age,cause,metric,year,val,upper,lower,country,state,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,snow_depth_mm,avg_wind_dir_deg,avg_wind_speed_kmh,peak_wind_gust_kmh,avg_sea_level_pres_hpa,sunshine_total_min,population,area,latitude,longitude
0,Prevalence,Taiwan,Male,85+ years,Migraine,Number,2011,5249.38,6903.76,3990.16,Taiwan,None,20.20,17.28,24.54,1244.90,0.00,NaN,NaN,NaN,NaN,NaN,23424615.00,36193.00,25.04,121.56


In [97]:
weather_migraine.isnull().sum()

measure                       0
location                      0
sex                           0
age                           0
cause                         0
metric                        0
year                          0
val                           0
upper                         0
lower                         0
country                       0
state                         0
avg_temp_c                10216
min_temp_c                 9329
max_temp_c                 9434
precipitation_mm           8880
snow_depth_mm              8880
avg_wind_dir_deg          28943
avg_wind_speed_kmh        26231
peak_wind_gust_kmh        57595
avg_sea_level_pres_hpa    38502
sunshine_total_min        56010
population                10250
area                       9557
latitude                   8880
longitude                  8880
dtype: int64

#### 5.3.2 Data Filtering

##### 5.3.2.1 Filter `combined_weather` by US Cities

In [ ]:
# Filter the combined weather data to only include the US
usa_weather = combined_weather[combined_weather['iso3'] == 'USA']

# Review the shape of the new dataframe
usa_weather.shape

In [ ]:
# View 10 rows of the new dataframe
usa_weather

In [ ]:
# Check for missing values
usa_weather.isnull().sum()

In [ ]:
# Check the unique values of the iso3 column, confirming no other countries are included
usa_weather['iso3'].unique()

#### 5.3.3 Data Cleaning (2nd Round)

##### 5.3.3.1 Drop Unnecessary Columns

After further review of the data, the `country` and `iso2` columns are no longer needed since we have filtered for iso3=USA, so we will drop them.

In [ ]:
# List of columns to keep
columns_to_keep = [col for col in usa_weather.columns if col not in ['country', 'iso2']]

# Use .loc to select only the columns to keep
usa_weather = usa_weather.loc[:, columns_to_keep]

In [ ]:
# Review the shape of the new dataframe
usa_weather.shape

##### 5.3.3.2 Drop Unnecessary Rows

In [ ]:
usa_weather.isnull().sum()

In [ ]:
# Drop rows with missing values
usa_weather = usa_weather.dropna(subset=['min_temp_c', 'max_temp_c', 'precipitation_mm'])

# Review the shape of the new dataframe
usa_weather.shape

In [ ]:
# Check for missing values
usa_weather.isnull().sum()

##### 5.3.3.3 Drop Duplicate Rows

In [ ]:
# Original DataFrame
usa_weather_row_count = len(usa_weather)

# DataFrame after dropping duplicates
usa_weather_deduplicated = usa_weather.drop_duplicates()
deduplicated_row_count = len(usa_weather_deduplicated)

# Calculate the number of rows that would be dropped
rows_to_be_dropped = usa_weather_row_count - deduplicated_row_count

# Print the difference
print(f"Rows to be dropped: {rows_to_be_dropped}")

#### 5.3.4 Preliminary Analysis

##### 5.3.4.1 Correlation Analysis for Weather Features

In [ ]:
weather_features = ['avg_temp_c', 'min_temp_c', 'max_temp_c', 'precipitation_mm', 'snow_depth_mm', 'avg_wind_dir_deg',\
                     'avg_wind_speed_kmh', 'peak_wind_gust_kmh', 'avg_sea_level_pres_hpa', 'sunshine_total_min']
correlation_matrix_usa_weather = usa_weather[weather_features].corr()

import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix_usa_weather, annot=True, cmap="coolwarm", vmin=-1, vmax=1)
plt.show()

The `sunshine_total_min` column has a lot of missing values, has a very weak correlation (-0.0066) with `avg_sea_level_pres_hpa`, and is not a focal point of this analysis, so we will drop that column.

In [ ]:
# Columns to keep
columns_to_keep = [col for col in usa_weather.columns if col not in ['sunshine_total_min']]

# Use .loc to select only the columns to keep
usa_weather = usa_weather.loc[:, columns_to_keep]

# Check for missing values
usa_weather.isnull().sum()

### 5.4 Handling Missing Values

#### 5.4.1 Non-pressure-related Columns

##### 5.4.1.1 Average Temperature Interpolation and Validation

For the `avg_temp_c` missing values, we will calculate the average of the `min_temp_c` and `max_temp_c` columns and use that value to fill in the missing average temperature values. A new, temporary column will be created called `avg_temp_c_interpolated` to hold these values.

In [ ]:
# Create a copy of the dataframe
usa_weather = usa_weather.copy()

# Create a column for calculating the `avg_temp_c` using the `min_temp_c` and `max_temp_c` columns
usa_weather['avg_temp_c_interpolated'] = usa_weather['avg_temp_c'].combine_first\
    ((usa_weather['min_temp_c'] + usa_weather['max_temp_c']) / 2)

Utilize the mean absolute error (MAE) to determine the accuracy of the interpolated values.

In [ ]:
# Import mean_absolute_error from sklearn.metrics
from sklearn.metrics import mean_absolute_error

# Filter out rows where either of the two columns is NaN
filtered_df = usa_weather.dropna(subset=['avg_temp_c', 'avg_temp_c_interpolated'])

# Calculate mean absolute error
mae = mean_absolute_error(filtered_df['avg_temp_c'], filtered_df['avg_temp_c_interpolated'])

# Print the mean absolute error
print(f"Mean Absolute Error: {mae}")


As noted above, the MAE is 0.0, so we will use the interpolated values to fill in the missing values for the `avg_temp_c` column. We will then drop the `avg_temp_c_interpolated` column. Please note after further investigation, it was found that the original `avg_temp_c` values are precisely calculated as the average of `min_temp_c` and `max_temp_c` values, so there is no loss of information.

Replace the missing values in the `avg_temp_c` column with the values from the `avg_temp_c_interpolated` column.

In [ ]:
# Fill missing values in the 'avg_temp_c' column with the average of the 
# 'min_temp_c' and 'max_temp_c' columns
usa_weather['avg_temp_c'] = usa_weather['avg_temp_c_interpolated']

Drop the `avg_temp_c_interpolated` column and check for any remaining missing values.

In [ ]:
# Drop the 'avg_temp_c_interpolated' column
usa_weather.drop(columns=['avg_temp_c_interpolated'], inplace=True)

# Check for missing values
usa_weather.isnull().sum()

In [ ]:
# Print the shape of the dataframe
print(f"Original Shape: {usa_weather.shape}")

##### 5.4.1.2 Aggregate Weather Data by Year and State

The migraine data is aggregated at an annual level and broken down by state, so we need to aggregate the weather data to match. A mean aggregation will be used for all columns except for the `precipitation_mm` and `snow_depth_mm` columns, which will be aggregated using a sum.

1. **Group by Year, State, and City Name**: Use pandas' `groupby` method to group data by both the `year`, `state`, and `city` columns.
2. **Aggregation Functions**: 
    - For temperatures (`avg_temp_c`, `min_temp_c`, `max_temp_c`), the mean is calculated for each year and state.
    - For wind (`avg_wind_dir_deg`, `avg_wind_speed_kmh`, `peak_wind_gust_kmh`), the mean is calculated for each year and state.
    - For `precipitation_mm` and `snow_depth_mm`, the total sum is calculated for each year and state.
    - For `avg_sea_level_pres_hpa`, the mean is calculated, assuming it's relevant to have an annual mean sea level pressure for each state.
3. **Spatial Data**: For latitude and longitude, the first observed value for each year and state is taken, assuming that these values are consistent within each state and year.

By following this methodology, the daily weather data is transformed into an annual summary by state, making it directly comparable with the annual, state-level migraine data for further analysis.

In [ ]:
# Group by 'year' and 'state', then aggregate the numerical columns
annual_usa_weather_by_stateCity = usa_weather.groupby(['year', 'state', 'city_name']).agg({
    'avg_temp_c': 'mean',
    'min_temp_c': 'mean',
    'max_temp_c': 'mean',
    'precipitation_mm': 'sum',
    'snow_depth_mm': 'sum',
    'avg_wind_dir_deg': 'mean',
    'avg_wind_speed_kmh': 'mean',
    'peak_wind_gust_kmh': 'mean',
    'avg_sea_level_pres_hpa': 'mean',
    'latitude': 'first',  # Assuming all latitudes are the same for a given year and state
    'longitude': 'first'  # Assuming all longitudes are the same for a given year and state
}).reset_index()

# Review the shape of the new dataframe
annual_usa_weather_by_stateCity.shape

In [ ]:
annual_usa_weather_by_stateCity.isnull().sum()

##### 5.4.1.3 Drop Wind Gust and Wind Direction Columns

In [ ]:
annual_usa_weather_by_stateCity.drop(columns=['peak_wind_gust_kmh', 'avg_wind_dir_deg'], inplace=True)

annual_usa_weather_by_stateCity.shape

In [ ]:
annual_usa_weather_by_stateCity.isnull().sum()

##### 5.4.1.4 Linear Interpolation for Average Wind Speed

In [ ]:
# Handle missing values for the `avg_wind_speed_kmh` column utilizing linear interpolation
annual_usa_weather_by_stateCity['avg_wind_speed_kmh'].interpolate(method='linear', inplace=True)

# Check for missing values
annual_usa_weather_by_stateCity.isnull().sum()

In [ ]:
# Check the shape of the dataframe
annual_usa_weather_by_stateCity.shape

#### 5.4.2 Pressure-related Columns

Sea level pressure can vary greatly depending on the location of the city and the main focus of this analysisis is to see if there is any correlation between sudden changes in sea level pressure and migraines.  As a result, we will not fill in missing values for the `avg_sea_level_pres_hpa` column without further research.  We will work through four different scenarios to determine which seems most accurate for this situation.
- Scenario #1: Leave/drop missing values for the `avg_sea_level_pres_hpa` value    
- Scenario #2: Utilize linear interpolation to fill in missing values for the `avg_sea_level_pres_hpa` column
- Scenario #3: Utilize forward fill to fill in missing values for the `avg_sea_level_pres_hpa` column
- Scenario #4: Utilize backward fill to fill in missing values for the `avg_sea_level_pres_hpa` column

##### 5.4.2.1 Leaving/Dropping Missing Values (Scenario #1)

In [ ]:
# Descriptive statistics for the `avg_sea_level_pres_hpa` column
annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa'].describe()

##### 5.4.2.2 Linear Interpolation (Scenario #2)

In [ ]:
# Handle missing values for `avg_sea_level_pres_hpa` column utilizing linear interpolation
annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa_linear'] = annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa'].interpolate(method='linear')

# Check for missing values
annual_usa_weather_by_stateCity.isnull().sum()

##### 5.4.2.3 Forward Fill (Scenario #3)

In [ ]:
# Handle missing values for `avg_sea_level_pres_hpa` column utilizing forward fill
annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa_ffill'] = annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa'].ffill()

# Check for missing values
annual_usa_weather_by_stateCity.isnull().sum()

##### 5.4.2.4 Backward Fill (Scenario #4)

In [ ]:
# Handle missing values for `avg_sea_level_pres_hpa` column utilizing backward fill
annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa_bfill'] = annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa'].bfill()

# Check for missing values
annual_usa_weather_by_stateCity.isnull().sum()

In [ ]:
annual_usa_weather_by_stateCity.describe()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

# Define common x and y limits
x_limits = [1005, 1025]  # Replace with the min and max values across all datasets for the x-axis
y_limits = [0, 125]  # Replace with the max frequency across all datasets for the y-axis

# Calculate number of bins for each dataset using the Square Root Rule
num_bins1 = int(np.sqrt(len(annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa'].dropna())))
num_bins2 = int(np.sqrt(len(annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa_linear'].dropna())))
num_bins3 = int(np.sqrt(len(annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa_ffill'].dropna())))
num_bins4 = int(np.sqrt(len(annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa_bfill'].dropna())))

# Plot each histogram on a different subplot
axes[0, 0].hist(annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa'].dropna(), bins=num_bins1, color='blue')
axes[0, 0].set_title('Original')
axes[0, 0].set_xlim(x_limits)
axes[0, 0].set_ylim(y_limits)

axes[0, 1].hist(annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa_linear'].dropna(), bins=num_bins2, color='green')
axes[0, 1].set_title('Linear Interpolated')
axes[0, 1].set_xlim(x_limits)
axes[0, 1].set_ylim(y_limits)

axes[1, 0].hist(annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa_ffill'].dropna(), bins=num_bins3, color='red')
axes[1, 0].set_title('Forward Fill')
axes[1, 0].set_xlim(x_limits)
axes[1, 0].set_ylim(y_limits)

axes[1, 1].hist(annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa_bfill'].dropna(), bins=num_bins4, color='purple')
axes[1, 1].set_title('Backward Fill')
axes[1, 1].set_xlim(x_limits)
axes[1, 1].set_ylim(y_limits)

for ax in axes.flat:
    ax.set(xlabel='Sea Level Pressure (hPa)', ylabel='Frequency')
    ax.set_xlim(x_limits)
    ax.set_ylim(y_limits)

# Display all subplots
plt.tight_layout()
plt.show()


##### 5.4.2.5 Decision on Filling Missing Values

After a thorough review of all four scenarios, we've decided to employ **Scenario #2 (linear interpolation)** for filling the missing values in the `avg_sea_level_pres_hpa` column. The rationale behind this choice is manifold:

- **Representation of Data**: Linear interpolation provides a smoother distribution than the other methods. This approach does not heavily skew the tail ends of the distribution, ensuring a more natural representation.
  
- **Preservation of Original Distribution**: Linear interpolation appears to retain the original data distribution more faithfully when filling in missing values, without introducing any discernible bias towards specific values.

- **Percentage of Missing Values**: With only 138 missing values, which represents 9.47% of the total count of 1457, the sheer accuracy of the method is not as paramount as it would be with a more substantial portion of missing values. Nevertheless, it's essential to utilize a method that delivers reliability, and linear interpolation does just that.

**Analysis of Alternative Methods:**

- The **forward-fill method**, though commendable, might introduce bias as it overlooks subsequent values after a missing point. It stands as our second preference.
  
- The **backward-fill method** is our third choice. While it does consider subsequent data points, its accuracy seems to trail the forward-fill method.
  
- Lastly, simply **leaving or dropping missing values** is the least appealing choice, as it disregards the rest of the dataset's information.


##### 5.4.2.6 Linear Interpolation for Average Sea Level Pressure

In [ ]:
# copy the dataframe
annual_usa_weather_by_stateCity = annual_usa_weather_by_stateCity.copy()

# Fill missing values in the `avg_sea_level_pres_hpa` column with the linear values
annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa'] = annual_usa_weather_by_stateCity['avg_sea_level_pres_hpa_linear']

# Drop the `avg_sea_level_pres_hpa_ffill` and `avg_sea_level_pres_hpa_bfill` columns
annual_usa_weather_by_stateCity.drop(columns=['avg_sea_level_pres_hpa_ffill', 'avg_sea_level_pres_hpa_bfill'], inplace=True)

# check for missing values
annual_usa_weather_by_stateCity.isnull().sum()

In [ ]:
# Filter out rows where either of the two columns is NaN
filtered_df = annual_usa_weather_by_stateCity.dropna(subset=['avg_sea_level_pres_hpa', 'avg_sea_level_pres_hpa_linear'])

# Calculate mean absolute error
mae = mean_absolute_error(filtered_df['avg_sea_level_pres_hpa'], filtered_df['avg_sea_level_pres_hpa_linear'])

# Print the mean absolute error
print(f"Mean Absolute Error: {mae}")

In [ ]:
annual_usa_weather_by_stateCity.drop(columns=['avg_sea_level_pres_hpa_linear'], inplace=True)

annual_usa_weather_by_stateCity.isnull().sum()

In [ ]:
annual_usa_weather_by_stateCity.head()

In [ ]:
annual_usa_weather_by_stateCity.shape

### 5.6 Join Migraine Data with Weather Data

Now that the weather data has been aggregated to match the migraine data, we can join the two datasets together. These datasets will be joined on the `year` and `state` columns from both the USA weather data and the migraine data.

In [ ]:
filtered_migraine_data.head()

In [ ]:
filtered_migraine_data.shape

In [ ]:
# Combine USA combined weather with migraine data
usa_weather_migraine = annual_usa_weather_by_stateCity.merge(filtered_migraine_data, 
                                      how='left', 
                                      left_on=['year', 'state'], 
                                      right_on=['year', 'state']
                                      )

# Review the shape of the new dataframe
usa_weather_migraine.shape

## 6. Feature Engineering

Discussing any new features that were created and why they were created. Also, discuss any features that were dropped and why they were dropped.

### 6.1 New Features

#### 6.1.1 Converting Celsius to Fahrenheit

##### 6.1.1.1 Convert `avg_temp_c` to `avg_temp_f`

In [ ]:
from temp_conversion import celsius_to_fahrenheit

usa_weather_migraine['avg_temp_f'] = usa_weather_migraine['avg_temp_c'].apply(celsius_to_fahrenheit)

##### 6.1.1.2 Convert `min_temp_c` to `min_temp_f`

In [ ]:
usa_weather_migraine['min_temp_f'] = usa_weather_migraine['min_temp_c'].apply(celsius_to_fahrenheit)

##### 6.1.1.3 Convert `max_temp_c` to `max_temp_f`

In [ ]:
usa_weather_migraine['max_temp_f'] = usa_weather_migraine['max_temp_c'].apply(celsius_to_fahrenheit)

##### 6.1.1.4 Reorder Temperature Columns

In [ ]:
# Reorder temperature columns
temp_col = usa_weather_migraine.pop('avg_temp_f')

# Insert columns at new position
usa_weather_migraine.insert(3, 'avg_temp_f', temp_col)

In [ ]:
# Reorder temperature columns
temp_col1 = usa_weather_migraine.pop('min_temp_f')

# Insert columns at new position
usa_weather_migraine.insert(4, 'min_temp_f', temp_col1)

In [ ]:
# Reorder temperature columns
temp_col2 = usa_weather_migraine.pop('max_temp_f')

# Insert columns at new position
usa_weather_migraine.insert(5, 'max_temp_f', temp_col2)

In [ ]:
usa_weather_migraine

##### 6.1.1.5 Convert `precipitation_mm` to `precipitation_in`

In [ ]:
from temp_conversion import mM_to_inches

usa_weather_migraine['precipitation_in'] = usa_weather_migraine['precipitation_mm'].apply(mM_to_inches)


In [ ]:
# Reorder temperature columns
temp_col3 = usa_weather_migraine.pop('precipitation_in')

# Insert columns at new position
usa_weather_migraine.insert(9, 'precipitation_in', temp_col3)

In [ ]:
usa_weather_migraine.head()

#### 6.1.2 Sea Level Pressure Missing Values

- `avg_sea_level_pres_hpa`: original column with missing values
- `avg_sea_level_pres_hpa_linear`: calculated by using linear interpolation to fill in missing values for the average sea level pressure
- `avg_sea_level_pres_hpa_ffill`: calculated by using forward fill to fill in missing values for the average sea level pressure
- `avg_sea_level_pres_hpa_bfill`: calculated by using backward fill to fill in missing values for the average sea level pressure

### 6.2 Dropped Features

In [ ]:
# usa_weather_migraine.drop(columns=['avg_temp_c', 'min_temp_c', 'max_temp_c'], inplace=True)

# usa_weather_migraine

In [ ]:
# usa_weather_migraine.drop(columns='precipitation_mm', inplace=True)

# usa_weather_migraine

## 7. Summary

Summarize the data preprocessing steps that were taken in this notebook.

## 8. Next Steps

Discuss any next steps that should be taken in the data analysis process/modeling phases.